In [9]:
from agents import ItemHelpers, Runner
from agents.model_settings import ModelSettings

from builder import AgentBuilder

In [10]:
mcp_manager = AgentBuilder.get_mcp_manager(
    tool_names=["location", "weather"],
    user_id="1234"
)

In [11]:
async def run(query):
    async with mcp_manager:
        agent = AgentBuilder.build(
            name="WeatherAgent",
            mcp_manager=mcp_manager,
            model_settings=ModelSettings(
                store=False,
                parallel_tool_calls=True
            ),
            user_name="id4thomas"
        )
        result = await Runner.run(agent, input=query)
    return result

In [12]:
query="내 현재 위치와 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘."
# query="서울 주변 도시들을 알려주고 그 도시들의 날씨들도 알려줘. 도시 명칭은 반환값 그대로 사용해"
# query="도구를 사용해서 서울 날씨 알려줘. 도구의 결과도 json 그대로 알려줘"

In [13]:
result = await run(query)
print(result)

RunResult:
- Last agent: Agent(name="WeatherAgent", ...)
- Final output (str):
    현재 날씨 정보를 확인했습니다:
    
    - **부산**: 6°C, 구름 조금, 습도 58%, 풍속 18km/h
    - **김해**: 7°C, 맑음, 습도 57%, 풍속 12km/h
    - **창원**: 7°C, 구름 조금, 습도 58%, 풍속 11km/h
    - **울산**: 7°C, 흐림, 습도 59%, 풍속 16km/h
    
    모든 도시의 날씨 정보를 확인하셨습니다.
- 16 new item(s)
- 4 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [14]:
for item in result.new_items:
    print(item.type)
    if item.type == "tool_call_item":
        raw = item.raw_item
        # MCP 툴인 경우
        tool_name = getattr(raw, "name", None) or getattr(raw, "tool_name", None)
        arguments = getattr(raw, "arguments", None)
        print(f"[tool_call] {tool_name}  args={arguments}")

    elif item.type == "tool_call_output_item":
        print(f"[tool_result] {item.output}")

    elif item.type == "message_output_item":
        print(f"[message] {ItemHelpers.text_message_output(item)}")

print(f"\n[final] {result.final_output}")

message_output_item
[message] 알겠습니다. 먼저 현재 위치를 확인하고, 주변 도시들을 확인한 후 그 도시들의 날씨 정보를 알려드리겠습니다.

tool_call_item
[tool_call] get_user_location  args={}
tool_call_output_item
[tool_result] {'type': 'text', 'text': '{"name":"Busan","coordinate":{"lat":35.1796,"lon":129.0756}}'}
message_output_item
[message] 현재 위치는 부산입니다. 부산 주변의 도시들을 확인해보겠습니다.


tool_call_item
[tool_call] get_nearby_cities  args={"city_name": "Busan"}
tool_call_output_item
[tool_result] {'type': 'text', 'text': '[{"name":"Gimhae","coordinate":{"lat":35.2285,"lon":128.8894}},{"name":"Changwon","coordinate":{"lat":35.228,"lon":128.6811}},{"name":"Ulsan","coordinate":{"lat":35.5384,"lon":129.3114}}]'}
message_output_item
[message] 부산 주변의 도시로는 김해, 창원, 울산이 있습니다. 각 도시의 날씨 정보를 확인해보겠습니다.


tool_call_item
[tool_call] get_current_weather  args={"city_name": "Busan"}
tool_call_item
[tool_call] get_current_weather  args={"city_name": "Gimhae"}
tool_call_item
[tool_call] get_current_weather  args={"city_name": "Changwon"}
tool_call_item
[to

# Streaming

In [15]:
from openai.types.responses import ResponseTextDeltaEvent

async def run_stream(query):
    async with mcp_manager:
        agent = AgentBuilder.build(
            name="WeatherAgent",
            mcp_manager=mcp_manager,
            model_settings=ModelSettings(
                store=False,
                parallel_tool_calls=True
            ),
            user_name="id4thomas"
        )
        current_text = ""
        in_tool_call = False

        result = Runner.run_streamed(agent, input=query)

        async for event in result.stream_events():

            if event.type == "raw_response_event":
                if isinstance(event.data, ResponseTextDeltaEvent):
                    if not in_tool_call:
                        delta = event.data.delta
                        current_text += delta
                        print(delta, end="", flush=True)

            elif event.type == "run_item_stream_event":
                item = event.item

                if item.type == "tool_call_item":
                    in_tool_call = True
                    if current_text:
                        print()
                        current_text = ""

                    raw = item.raw_item
                    tool_name = getattr(raw, "name", None) or getattr(raw, "tool_name", None)
                    arguments = getattr(raw, "arguments", None)
                    print(f"\n🔧 [tool_call] {tool_name}({arguments})")

                elif item.type == "tool_call_output_item":
                    in_tool_call = False
                    print(f"   ↳ [tool_result] {item.output}\n")

                elif item.type == "message_output_item":
                    # 텍스트 스트림이 끝났음을 표시
                    if current_text:
                        print()
                        current_text = ""
                    print("─" * 40)

            elif event.type == "agent_updated_stream_event":
                print(f"\n🤖 [agent] {event.new_agent.name}\n")
                
            else:
                print("EVENT", event.type)

        print(f"\n✅ [final]\n{result.final_output}")

In [16]:
await run_stream(query)


🤖 [agent] WeatherAgent

알겠습니다. 먼저 현재 위치를 확인하고, 주변 도시들을 확인한 후 그 도시들의 날씨를 알려드리겠습니다.


🔧 [tool_call] get_user_location()
────────────────────────────────────────
   ↳ [tool_result] {'type': 'text', 'text': '{"name":"Busan","coordinate":{"lat":35.1796,"lon":129.0756}}'}

현재 위치는 부산입니다. 주변 도시들을 확인해 보겠습니다.



🔧 [tool_call] get_nearby_cities({"city_name": "Busan"})
────────────────────────────────────────
   ↳ [tool_result] {'type': 'text', 'text': '[{"name":"Gimhae","coordinate":{"lat":35.2285,"lon":128.8894}},{"name":"Changwon","coordinate":{"lat":35.228,"lon":128.6811}},{"name":"Ulsan","coordinate":{"lat":35.5384,"lon":129.3114}}]'}

부산 주변 도시로는 김해, 창원, 울산이 있습니다. 각 도시의 날씨를 확인해 드리겠습니다.







🔧 [tool_call] get_current_weather({"city_name": "Busan"})

🔧 [tool_call] get_current_weather({"city_name": "Gimhae"})

🔧 [tool_call] get_current_weather({"city_name": "Changwon"})

🔧 [tool_call] get_current_weather({"city_name": "Ulsan"})
────────────────────────────────────────
   ↳ [tool_result] {'typ